<a href="https://colab.research.google.com/github/rohank36/jackie-2.0/blob/main/jackie_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install gradientai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.6/353.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.0 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.4
    Uninstalling pydantic-2.6.4:
      Successfully uninstalled pydantic-2.6.4


In [22]:
import os
os.environ['GRADIENT_ACCESS_TOKEN'] = ""
os.environ['GRADIENT_WORKSPACE_ID'] = ""

In [5]:
import csv

from gradientai import Gradient

path_to_csv_file = "/content/nytcrosswords.csv"

In [7]:
print("Parsing data...")
rows_to_keep = []
with open(path_to_csv_file, "r", encoding="utf-8-sig", errors="replace") as f:
    reader = csv.DictReader(f, delimiter=",")
    for row in reader:
        # Extracting input (clue) and output (word) from the row
        clue = row["Clue"]
        word = row["Word"]
        rows_to_keep.append({"clue": clue, "word": word})

lines = []
for row in rows_to_keep:
    clue = row["clue"].replace('"', '\\"')
    word = row["word"].replace('"', '\\"')

    # Constructing the input/output sequence
    start_str = "<s>### Input:\n"
    mid_str = '''\n\n### Response:\n'''
    end_str = '''</s>'''
    total_line = start_str + clue + mid_str + word + end_str
    obj = {
        "inputs": total_line
    }
    lines.append(obj)

print(f"Generated {len(lines)} lines to fine-tune")
print(f"Example training line: {lines[0]}")

lines_per_chunk = 20
all_chunks = []
for line in lines:
    if len(all_chunks) == 0 or len(all_chunks[-1]) == lines_per_chunk:
        all_chunks.append([])
    all_chunks[-1].append(line)

# Limiting to 100 chunks
all_chunks = all_chunks[:100]

print(f"\nFine-tuning model adapter")
gradient = Gradient()
base = gradient.get_base_model(base_model_slug="nous-hermes2")
my_adapter = base.create_model_adapter(name="rohanbot")
print(f"Created model with ID {my_adapter.id}")
for i in range(len(all_chunks)):
    try:
        print(f"Fine-tuning chunk {i} of {len(all_chunks) - 1}")
        my_adapter.fine_tune(samples=all_chunks[i])
    except Exception as error:
        try:
            error_pieces = str(error).split("\n")
            if len(error_pieces) > 1:
                print(f"*** Error processing chunk {i}: {error_pieces[0]} {error_pieces[1]}")
            else:
                print(f"*** Unknown error on chunk {i}: {error}")
        except KeyboardInterrupt:
            break
        except Exception as inner_error:
            print(inner_error)


Parsing data...
Generated 781573 lines to fine-tune
Example training line: {'inputs': '<s>### Input:\nAction done while saying \\"Good dog\\"\n\n### Response:\nPAT</s>'}

Fine-tuning model adapter
Created model with ID ada699a3-172f-41fd-8b79-24261985be00_model_adapter
Fine-tuning chunk 0 of 99
Fine-tuning chunk 1 of 99
Fine-tuning chunk 2 of 99
Fine-tuning chunk 3 of 99
Fine-tuning chunk 4 of 99
Fine-tuning chunk 5 of 99
Fine-tuning chunk 6 of 99
Fine-tuning chunk 7 of 99
Fine-tuning chunk 8 of 99
Fine-tuning chunk 9 of 99
Fine-tuning chunk 10 of 99
Fine-tuning chunk 11 of 99
Fine-tuning chunk 12 of 99
Fine-tuning chunk 13 of 99
Fine-tuning chunk 14 of 99
Fine-tuning chunk 15 of 99
Fine-tuning chunk 16 of 99
Fine-tuning chunk 17 of 99
Fine-tuning chunk 18 of 99
Fine-tuning chunk 19 of 99
Fine-tuning chunk 20 of 99
Fine-tuning chunk 21 of 99
Fine-tuning chunk 22 of 99
Fine-tuning chunk 23 of 99
Fine-tuning chunk 24 of 99
Fine-tuning chunk 25 of 99
Fine-tuning chunk 26 of 99
Fine-tuning

In [ ]:
# if your colab instance gets deleted, you can run this to get the model names
gradient = Gradient()
# if necessary, go back and find your previously created models and their IDs
old_models = gradient.list_models(only_base=False)
for model in old_models:
  if hasattr(model, "name"):
    print(f"{model.name}: {model.id}")

In [19]:
role_play_prompt = "You are an elite crossword bot"
query = "One of 14 on a roly-poly bug"
templated_query = f"<s>### Instruction:\n{role_play_prompt}\n\n###Input:\n{query}\n\n### Response:\n"
response = my_adapter.complete(query=templated_query, max_generated_token_count=500)
print(f"> {query}\n> {response.generated_output}\n\n")


> One of 14 on a roly-poly bug
> TENPIN




In [21]:
my_adapter.delete()

NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'strict-transport-security': 'max-age=63072000; includeSubDomains; preload', 'referrer-policy': 'no-referrer', 'x-content-type-options': 'nosniff', 'x-download-options': 'noopen', 'x-frame-options': 'SAMEORIGIN', 'x-xss-protection': '1; mode=block', 'content-type': 'application/json; charset=utf-8', 'etag': '"cnneatxz1cu"', 'vary': 'Accept-Encoding', 'X-Cloud-Trace-Context': '82f082f2dff43a19b291a6f5f4147d9a;o=1', 'Date': 'Thu, 11 Apr 2024 03:08:28 GMT', 'Server': 'Google Frontend', 'Content-Length': '30', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"message":"Model not found."}
